# Recommending Other Songs by Lyrics

In [1]:
import pandas as pd
import numpy as np
import json
import ast
from sklearn.neighbors import NearestNeighbors

## Use Pretrained GloVe Model to Vectorize Lyrics
NOTE: Due to the size of the file 'glove.6B.50d.txt', it is not saved in this repo. This file can be obtained from [Stanford's GloVe Website](https://nlp.stanford.edu/projects/glove/)

In [2]:
def loadGloveModel():
    print("Loading Glove Model")
    f = open('glove.6B.50d.txt','r')
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print(len(gloveModel)," words loaded!")
    return gloveModel

gloveModel=loadGloveModel()

Loading Glove Model
400000  words loaded!


In [3]:
# Import data and evaluate strings as lists for clean_lyrics
rec_df = pd.read_csv('recommendation_resources/preprocessed_lyrics.csv', index_col=0)
rec_df.clean_lyrics = rec_df.clean_lyrics.apply(lambda x: ast.literal_eval(x))
rec_df.head()

,artist,song_name,file_name,lyrics,clean_lyrics
0,Gary Wright,Love Is Alive,love_is_alive.txt,"Well, I think it's time to get ready To realiz...","[well, think, time, get, ready, realize, find,..."
1,LMFAO,Party Rock Anthem,party_rock_anthem.txt,Party rock Let's go! Party rock is in the hou...,"[party, rock, let, us, go, party, rock, house,..."
2,Metallica,2 X 4,2_x_4.txt,"I’m gonna make you, shake you, take you I’m go...","[go, make, shake, take, go, one, break, put, s..."
3,Metallica,Prince Charming,prince_charming.txt,There’s a black cloud overhead That’s me And t...,"[black, cloud, overhead, poison, ivy, choke, t..."
4,Metallica,Now That We'Re Dead,now_that_we're_dead.txt,When darkness falls May it be That we should s...,"[darkness, fall, may, see, light, reaper, call..."


In [4]:
# Create column for GloVe vectors. Use zero arrays for words not in GloVe.
def get_vector(word):
    try:
        return gloveModel[word]
    except:
        return np.zeros(50)

# For simple KNN, take average of vectorized words in cleaned lyrics
rec_df['vectors'] = rec_df.clean_lyrics.apply(lambda x: np.mean([get_vector(word) for word in x], axis=0))
rec_df.head()

,artist,song_name,file_name,lyrics,clean_lyrics,vectors
0,Gary Wright,Love Is Alive,love_is_alive.txt,"Well, I think it's time to get ready To realiz...","[well, think, time, get, ready, realize, find,...","[0.19744645945945963, 0.37155174594594587, 0.2..."
1,LMFAO,Party Rock Anthem,party_rock_anthem.txt,Party rock Let's go! Party rock is in the hou...,"[party, rock, let, us, go, party, rock, house,...","[-0.07758194069264084, 0.05038790173160177, 0...."
2,Metallica,2 X 4,2_x_4.txt,"I’m gonna make you, shake you, take you I’m go...","[go, make, shake, take, go, one, break, put, s...","[0.23941783802816885, 0.06187164084507043, 0.0..."
3,Metallica,Prince Charming,prince_charming.txt,There’s a black cloud overhead That’s me And t...,"[black, cloud, overhead, poison, ivy, choke, t...","[0.09181784571428582, 0.24025833714285713, 0.0..."
4,Metallica,Now That We'Re Dead,now_that_we're_dead.txt,When darkness falls May it be That we should s...,"[darkness, fall, may, see, light, reaper, call...","[0.26997853366336627, 0.3395790495049503, 0.02..."


# Generate Recommendations with K-Nearest Neighbors

In [5]:
# Default settings for KNN will suffice for this model
knn = NearestNeighbors()
knn.fit(list(rec_df.vectors))

# Get top 6 nearest neighbors
recos = knn.kneighbors(list(rec_df.vectors), 6, return_distance=False)
recos = [rec[1:] for rec in recos] # Drop first neighbor because self
recos[:5]

[array([1555, 1721, 1972,  691,   30]),
 array([1491,  438,  737, 1223, 1638]),
 array([1465,  446,  391,  227, 1229]),
 array([1554,  889,  248,  870, 1217]),
 array([2077,  824, 1969, 1484, 1838])]

In [6]:
# Get details of recommendations based on reco id's
all_recs = []
for i in rec_df.index:
    
    rec_ids = recos[i]
    
    recs = {}
    for j in rec_ids:
        recs[rec_df.artist[j]]={rec_df.song_name[j]:rec_df.file_name[j]}
    
    all_recs.append(recs)

rec_df['recommendations'] = all_recs
rec_df.head()

,artist,song_name,file_name,lyrics,clean_lyrics,vectors,recommendations
0,Gary Wright,Love Is Alive,love_is_alive.txt,"Well, I think it's time to get ready To realiz...","[well, think, time, get, ready, realize, find,...","[0.19744645945945963, 0.37155174594594587, 0.2...",{'Smokey Robinson & The Miracles': {'Cause I L...
1,LMFAO,Party Rock Anthem,party_rock_anthem.txt,Party rock Let's go! Party rock is in the hou...,"[party, rock, let, us, go, party, rock, house,...","[-0.07758194069264084, 0.05038790173160177, 0....",{'Outkast': {'A Life In The Day Of Benjamin An...
2,Metallica,2 X 4,2_x_4.txt,"I’m gonna make you, shake you, take you I’m go...","[go, make, shake, take, go, one, break, put, s...","[0.23941783802816885, 0.06187164084507043, 0.0...",{'Wilson Phillips': {'Hold On': 'hold_on.txt'}...
3,Metallica,Prince Charming,prince_charming.txt,There’s a black cloud overhead That’s me And t...,"[black, cloud, overhead, poison, ivy, choke, t...","[0.09181784571428582, 0.24025833714285713, 0.0...",{'Smokey Robinson & The Miracles': {'The Track...
4,Metallica,Now That We'Re Dead,now_that_we're_dead.txt,When darkness falls May it be That we should s...,"[darkness, fall, may, see, light, reaper, call...","[0.26997853366336627, 0.3395790495049503, 0.02...",{'Iron Maiden': {'Starblind': 'starblind.txt'}...


### Create JSON File with Recommendations

In [7]:
recs = {}
for artist in rec_df.artist.unique():
    recs[artist] = {}

for i in rec_df.index:
    song_info = {
        'file_name':rec_df.file_name[i],
        'recommendations':rec_df.recommendations[i],
    }
    recs[rec_df.artist[i]][rec_df.song_name[i]]=song_info

with open("recommendations.json", "w") as outfile:  
    json.dump(recs, outfile)

recs

{'Gary Wright': {'Love Is Alive': {'file_name': 'love_is_alive.txt',
   'recommendations': {'Smokey Robinson & The Miracles': {'Cause I Love You': 'cause_i_love_you.txt'},
    'Ray Charles': {"Jumpin' In The Morning": "jumpin'_in_the_morning.txt"},
    'Gregory Abbott': {'Shake You Down': 'shake_you_down.txt'},
    'KISS': {"Love'S A Deadly Weapon": "love's_a_deadly_weapon.txt"},
    'Bee Gees': {"Stayin' Alive": "stayin'_alive.txt"}}}},
 'LMFAO': {'Party Rock Anthem': {'file_name': 'party_rock_anthem.txt',
   'recommendations': {'Outkast': {'A Life In The Day Of Benjamin André (Incomplete)': 'a_life_in_the_day_of_benjamin_andré_(incomplete).txt'},
    'Blondie': {'The Beast': 'the_beast.txt'},
    'Talking Heads': {'What A Day That Was': 'what_a_day_that_was.txt'},
    'Def Leppard': {'Stand Up (Kick Love Into Motion)': 'stand_up_(kick_love_into_motion).txt'},
    'Drake': {'One Dance': 'one_dance.txt'}}}},
 'Metallica': {'2 X 4': {'file_name': '2_x_4.txt',
   'recommendations': {'Wil